<a href="https://colab.research.google.com/github/mampisarkar111/Matlab/blob/master/Wisonet_kernel_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L "https://rice.box.com/shared/static/bcoy3ob0dme3umpurqmf0p6o48bznkj1" -o SWING3_2006_subset.nc
!curl -L "https://rice.box.com/shared/static/uuy9m15qc1p7s4wm1yrzfzxc6knx7hzw" -o TES_monthly_5deg_strict.nc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     8    0     8    0     0      5      0 --:--:--  0:00:01 --:--:--  8000
100 12.7M  100 12.7M    0     0  6324k      0  0:00:02  0:00:02 --:--:--  129M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    10    0    10    0     0      8      0 --:--:--  0:00:01 --:--:-- 10000
100 3095M  100 3095M    0     0   103M      0  0:00:29  0:00:29 --:--:-- 43.1M


In [ ]:
!ls -lh
!pip install cartopy

total 3.1G
drwxr-xr-x 1 root root 4.0K Jun 12 13:36 sample_data
-rw-r--r-- 1 root root  13M Jun 14 16:25 SWING3_2006_subset.nc
-rw-r--r-- 1 root root 3.1G Jun 14 16:26 TES_monthly_5deg_strict.nc
  Using cached Cartopy-0.24.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 86.7 MB/s eta 0:00:00


In [33]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from cartopy.io import DownloadWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="xarray")
warnings.filterwarnings("ignore", category=DownloadWarning, module="cartopy")
warnings.filterwarnings("ignore", category=RuntimeWarning)

# === Load Datasets Once ===
ds = xr.open_dataset("SWING3_2006_subset.nc")
sat = xr.open_dataset("TES_monthly_5deg_strict.nc")

# === Common Dimensions ===
lon = ds['lon'].values
lat = ds['lat'].values
p = ds['p'].values
tes_lat = sat['lat'].values
tes_lon = sat['lon'].values
tes_p = sat['level'].values
tes_time = sat['time'].values

# === Widget Controls ===
plevel_input = widgets.BoundedFloatText(value=500, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')
log_toggle = widgets.Checkbox(value=True, description='Log X-axis')
step_dropdown = widgets.Dropdown(options=['All', 'Model True vs TES', 'Model True vs TES vs Apriori'],value='All',description='Step View:')
lat_min_input = widgets.BoundedFloatText(value=-20, min=-90, max=90, step=1, description='Lat Min:')
lat_max_input = widgets.BoundedFloatText(value=0, min=-90, max=90, step=1, description='Lat Max:')
lon_min_input = widgets.BoundedFloatText(value=-70, min=-180, max=180, step=5, description='Lon Min:')
lon_max_input = widgets.BoundedFloatText(value=-40, min=-180, max=180, step=5, description='Lon Max:')

# === Plot Function ===
def update_plot(month, plevel):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan


    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    HDO_vmr1_3d = q_vmr_3d * Rvsmow * (1 + dD / 1000)
    HDO_vmr1_3d = HDO_vmr1_3d[m, :, :, :]

    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))



    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    HDO_globe = sat['HDO_vmr'].values[t_tes, idx_tes, :, :]
    HDO_model = HDO_vmr1_3d[idx_mod, :, :]

    HDO_globe[HDO_globe == -999] = np.nan
    HDO_model[HDO_model == -999] = np.nan

    combined = np.concatenate([HDO_globe[np.isfinite(HDO_globe)], HDO_model[np.isfinite(HDO_model)]])
    vmin, vmax = np.nanpercentile(combined, 1), np.nanpercentile(combined, 99)
    levels = np.linspace(vmin, vmax, 21)


    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    HDO_model_sorted = HDO_model[:, sort_idx]

    fig = plt.figure(figsize=(14, 10))

    ax1 = fig.add_subplot(2, 1, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(tes_lon, tes_lat, HDO_globe, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3)
    ax1.set_title(f'TES HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs1, ax=ax1).set_label('TES HDO VMR')

    ax2 = fig.add_subplot(2, 1, 2, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(lon_plot_sorted, lat, HDO_model_sorted, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3)
    ax2.set_title(f'Model Ensemble HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs2, ax=ax2).set_label('Model Ensemble HDO VMR')






# === Display Widgets ===
ui = VBox([
    HBox([plevel_input]),
    HBox([month_slider]),
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider,
})
display(ui, out)

Output()

In [23]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from cartopy.io import DownloadWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="xarray")
warnings.filterwarnings("ignore", category=DownloadWarning, module="cartopy")
warnings.filterwarnings("ignore", category=RuntimeWarning)

# === Load Datasets Once ===
ds = xr.open_dataset("SWING3_2006_subset.nc")
sat = xr.open_dataset("TES_monthly_5deg_strict.nc")

# === Common Dimensions ===
lon = ds['lon'].values
lat = ds['lat'].values
p = ds['p'].values
tes_lat = sat['lat'].values
tes_lon = sat['lon'].values
tes_p = sat['level'].values
tes_time = sat['time'].values

# === Widget Controls ===
plevel_input = widgets.BoundedFloatText(value=500, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')
log_toggle = widgets.Checkbox(value=True, description='Log X-axis')
step_dropdown = widgets.Dropdown(options=['All', 'Model True vs TES', 'Model True vs TES vs Apriori'],value='All',description='Step View:')
lat_min_input = widgets.BoundedFloatText(value=-20, min=-90, max=90, step=1, description='Lat Min:')
lat_max_input = widgets.BoundedFloatText(value=0, min=-90, max=90, step=1, description='Lat Max:')
lon_min_input = widgets.BoundedFloatText(value=-70, min=-180, max=180, step=5, description='Lon Min:')
lon_max_input = widgets.BoundedFloatText(value=-40, min=-180, max=180, step=5, description='Lon Max:')

# === Plot Function ===
def update_plot(month, plevel, log_x, step_view, lat_min, lat_max, lon_min, lon_max):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan

    lat_mask = (lat >= lat_min) & (lat <= lat_max)
    # lon_min_wrapped = (lon_min + 360) % 360
    # lon_max_wrapped = (lon_max + 360) % 360
    # lon_mask = (lon >= lon_min_wrapped) & (lon <= lon_max_wrapped)

    lon_wrapped = (lon + 360) % 360
    lon_min_wrapped = (lon_min + 360) % 360
    lon_max_wrapped = (lon_max + 360) % 360

    if lon_min_wrapped < lon_max_wrapped:
        lon_mask = (lon_wrapped >= lon_min_wrapped) & (lon_wrapped <= lon_max_wrapped)
    else:
        # Handle wrap-around (e.g. 170°E to -170°W)
        lon_mask = (lon_wrapped >= lon_min_wrapped) | (lon_wrapped <= lon_max_wrapped)

    dD_eq = dD[m, :, lat_mask, :][:, :, lon_mask]
    q_eq = q[m, :, lat_mask, :][:, :, lon_mask]

    dD_mean = np.nanmean(np.nanmean(dD_eq, axis=2), axis=0)
    q_mean = np.nanmean(np.nanmean(q_eq, axis=2), axis=0)

    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528
    q_vmr = (q_mean / M_H2O) / ((q_mean / M_H2O) + ((1 - q_mean) / M_air))
    HDO_vmr1 = q_vmr * Rvsmow * (1 + dD_mean / 1000)

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    HDO_vmr1_3d = q_vmr_3d * Rvsmow * (1 + dD / 1000)
    HDO_vmr1_3d = HDO_vmr1_3d[m, :, :, :]

    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))

    tes_mask = (tes_lat >= lat_min) & (tes_lat <= lat_max)
    # tes_lon_wrapped = (tes_lon + 360) % 360
    # tes_lon_mask = (tes_lon_wrapped >= lon_min_wrapped) & (tes_lon_wrapped <= lon_max_wrapped)

    tes_lon_wrapped = (tes_lon + 360) % 360
    if lon_min_wrapped < lon_max_wrapped:
        tes_lon_mask = (tes_lon_wrapped >= lon_min_wrapped) & (tes_lon_wrapped <= lon_max_wrapped)
    else:
        tes_lon_mask = (tes_lon_wrapped >= lon_min_wrapped) | (tes_lon_wrapped <= lon_max_wrapped)

    HDO_ret = sat['HDO_vmr'].values[t_tes, :, tes_mask, :][:, :, tes_lon_mask]
    HDO_a = sat['HDO_ConstraintVector'].values[t_tes, :, tes_mask, :][:, :, tes_lon_mask]
    AK = sat['AK_HDO'].values[t_tes, :, :, tes_mask, :][:, :, :, tes_lon_mask]

    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    HDO_globe = sat['HDO_vmr'].values[t_tes, idx_tes, :, :]
    HDO_model = HDO_vmr1_3d[idx_mod, :, :]

    HDO_globe[HDO_globe == -999] = np.nan
    HDO_model[HDO_model == -999] = np.nan

    combined = np.concatenate([HDO_globe[np.isfinite(HDO_globe)], HDO_model[np.isfinite(HDO_model)]])
    vmin, vmax = np.nanpercentile(combined, 1), np.nanpercentile(combined, 99)
    levels = np.linspace(vmin, vmax, 21)

    HDO_ret[HDO_ret == -999] = np.nan
    HDO_a[HDO_a == -999] = np.nan
    AK[AK == -999] = np.nan

    HDO_ret_avg = np.nanmean(np.nanmean(HDO_ret, axis=2), axis=0)
    HDO_a_avg = np.nanmean(np.nanmean(HDO_a, axis=2), axis=0)
    AK_avg = np.nanmean(np.nanmean(AK, axis=3), axis=0)

    HDO_vmr1_interp = np.interp(tes_p, p[::-1], HDO_vmr1[::-1])
    ln_true = np.log(HDO_vmr1_interp)
    ln_apriori = np.log(HDO_a_avg)
    ln_smoothed = ln_apriori + AK_avg @ (ln_true - ln_apriori)
    # ln_smoothed = ln_apriori + AK_avg[:,3].T @ (ln_true - ln_apriori)
    HDO_recon = np.exp(ln_smoothed)

    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    HDO_model_sorted = HDO_model[:, sort_idx]

    fig = plt.figure(figsize=(14, 10))

    ax1 = fig.add_subplot(2, 2, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(tes_lon, tes_lat, HDO_globe, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3)
    ax1.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax1.set_title(f'TES HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs1, ax=ax1, orientation='horizontal', pad=0.05).set_label('TES HDO VMR')

    ax2 = fig.add_subplot(2, 2, 3, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(lon_plot_sorted, lat, HDO_model_sorted, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3)
    ax2.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax2.set_title(f'Model Ensemble HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs2, ax=ax2, orientation='horizontal', pad=0.05).set_label('Model Ensemble HDO VMR')

    ax3 = fig.add_subplot(2, 2, 2)
    if step_view == 'Model True vs TES':
        ax3.plot(HDO_vmr1_interp, tes_p, 'k-', label='Model True')
        ax3.plot(HDO_ret_avg, tes_p, 'b-o', label='TES Retrieved')
    elif step_view == 'Model True vs TES vs Apriori':
        ax3.plot(HDO_vmr1_interp, tes_p, 'k-', label='Model True')
        ax3.plot(HDO_ret_avg, tes_p, 'b-o', label='TES Retrieved')
        ax3.plot(HDO_a_avg, tes_p, 'g--', label='TES Apriori')
    else:
        ax3.plot(HDO_ret_avg, tes_p, 'b-o', label='TES Retrieved')
        ax3.plot(HDO_a_avg, tes_p, 'g--', label='TES Apriori')
        ax3.plot(HDO_vmr1_interp, tes_p, 'k-', label='Model True')
        ax3.plot(HDO_recon, tes_p, 'm-.', label='AK-applied')
    ax3.set_xlabel('HDO VMR [mol/mol]')
    ax3.set_ylabel('Pressure [hPa]')
    lat_label = f"{abs(lat_min):.0f}°{'S' if lat_min < 0 else 'N'}–{abs(lat_max):.0f}°{'S' if lat_max < 0 else 'N'}"
    lon_label = f"{abs(lon_min):.0f}°{'W' if lon_min < 0 else 'E'}–{abs(lon_max):.0f}°{'W' if lon_max < 0 else 'E'}"
    ax3.set_title(f'Vertical Profile ({lat_label}, {lon_label}, {user_date_str})')

    # Collect all VMR values that are not NaN
    all_profiles = [HDO_ret_avg, HDO_a_avg, HDO_vmr1_interp, HDO_recon]
    valid_values = np.concatenate([x[np.isfinite(x)] for x in all_profiles])

    x_min = max(np.nanmin(valid_values), 1e-10)
    x_max = np.nanmax(valid_values)
    ax3.invert_yaxis()
    ax3.grid(True)
    ax3.set_ylim(1000, 0)
    ax3.legend(loc='lower left')

    if log_x:
        ax3.set_xscale('log')
        ax3.set_xlim(1e-10, x_max * 1.1)  # Add a small margin
    else:
        ax3.set_xlim(0, x_max * 1.1)


    import matplotlib.cm as cm
    import matplotlib.colors as colors

    ax4 = fig.add_subplot(2, 2, 4)
    norm = colors.Normalize(vmin=np.min(tes_p), vmax=np.max(tes_p))
    cmap = cm.viridis
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    AK_avg=AK_avg.T
    for i in range(AK_avg.shape[0]):
        color = cmap(norm(tes_p[i]))
        ax4.plot(AK_avg[i, :], tes_p, color=color, label=f'Level {tes_p[i]:.0f} hPa')

    ax4.set_xlabel('AK Value (dx_est / dx_true)')
    ax4.set_ylabel('Pressure [hPa]')
    ax4.set_title('Averaging Kernel Rows (colored by level)')
    ax4.invert_yaxis()
    ax4.grid(True)

    # Add colorbar
    cbar = fig.colorbar(sm, ax=ax4, orientation='vertical', pad=0.02)
    cbar.set_label('Retrieved Level [hPa]')


# === Display Widgets ===
ui = VBox([
    HBox([plevel_input]),
    HBox([month_slider]),
    HBox([lat_min_input, lat_max_input, lon_min_input, lon_max_input]),
    HBox([log_toggle, step_dropdown])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider,
    'log_x': log_toggle,
    'step_view': step_dropdown,
    'lat_min': lat_min_input,
    'lat_max': lat_max_input,
    'lon_min': lon_min_input,
    'lon_max': lon_max_input
})
display(ui, out)

Output()